<a id="top"/>


### Come back to [Home](FinalProjectReport.ipynb)


# STOCK DATA PROCESSING

>The content of this section is:
>1. [Choosing the dataset](#choose) 
>1. [Obtain data](#obtain) 
>1. [Data processing](#proc) 
>1. [Dataset's stats](#stats)
>1. [Some graphs](#graphs)

>Here we import the modules that we will need in order to extract the data to manipulate them 

In [1]:
#!pip install yfinance
#!pip install yahoofinancials
import pandas as pd
import numpy as np
import pickle
import csv
from datetime import datetime, timedelta
import yfinance as yf
import yahoofinancials
from collections import Counter, defaultdict, deque
import json
import urllib

>In this section we perform the initial preprocessing of the data to a shape which is best for use in further analysis. We will show:
* where we obtain our data
* convert the data to a form that it will be easier to use

<a id="choose"/>

## Choosing the dataset
> We decided to include a number of financial instruments to create the most reliable and inclusive picture of the market. Our goal was to analyze the data initially, find irregularities and investigate the data on the basis of questions. The most interesting actions were nominated for further analysis.

> Our analysis includes multiple, different financial indexes and products. These include:
>
>
> * Exchange-traded funds (ETFs):
>  * What it is: An ETF is a investment fund traded in stock exchanges. ETFs allow exposure to a collection of assets in a specific market and can be good aggregators of the movement in a segment of that market
>  * Which did we get:
>    * iShares China Large-Cap ETF
>    * VanEck Vectors Africa ETF
>    * Franklin FTSE Italy ETF
>    * Xtrackers Harvest CSI 300 China A-Shares ETF
> * Stock Market Indexes:
>  * What it is: A Stock Market Index is an aggregation of the value of the traded assets in a market. Indexes give the overall action and sentiment in a market.
>  * Which did we get:
>    * OMX Stockhlom
>    * OMX Helsinki
>    * Oslo Børs All-share Index
>    * Bolsa de Madrid 35
>    * Dow Jones Industrial Average
>    * CAC 40 France
>    * DAX Frankfurt
>    * S&P 500
>    * Hang Seng Index Hong Kong
>    * Nikkei 225 Japan
>    * FTSE 100 London
> * Futures:
>  * What it is: A Futures contract is an agreement to buy or sell a specific asset at a predetermined price at a specifc time in the future. They are instruments that allow to gauge the future price of that asset and ascertain the direction of pricing.
>  * Which did we get:
>    * Oil Crude Jun 2020 Futures
>    * Nasdaq 100 June 2020 Futures
> * Forex Pairs:
>  * What it is: Foreign Exchange pairs are couples of currencies comprised of a base currency denominated in the quote currency. In a pair like CNY/EUR we see how many EURs are required to buy 1 CNY.
>  * Which did we get:
>    * CNY/EUR
>    * CNY/USD
> * Cryptocurrency pairs:
>  * What it is: Similar to forex pairs, cryptocurrency pairs work the same way but reveal the prices of digital assets of the likes of Bitcoin or Ethereum.
>  * Which did we get:
>    * Bitcoin/USD

<a id="obtain"/>

## Obtaining the Data
> We obtained 2.5 years of data using the [yahoo financials API](https://pypi.org/project/yahoo-finance/) for the below indexes (see below for the exact dates). The available data comprehend: 'High', 'Low', 'Close', 'Adj Close' and 'Volume' (with a day precision). In order to fullfill our purposes we just need 'Adj Close'.

In [2]:
STOCKS = {
  '^OMX': 'OMX Stockhlom',
  '^OMXH25': 'OMX Helsinki',
  '^OSEAX': 'Oslo Børs All-share Index',
  '^OMXCGI': 'OMX Copenhagen',
  'FXI': 'iShares China Large-Cap ETF',
  '^IBEX': 'Bolsa de Madrid 35',
  'AFK': 'VanEck Vectors Africa ETF',
  'BTC-USD': 'Bitcoin/USD',
  'CL=F': 'Oil Crude Jun 2020 Futures',
  'CNYEUR=X?P=CNYEUR=X': 'CNY/EUR',
  'CNYUSD=X?P=CNYUSD=X': 'CNY/USD',
  'FLIY': 'Franklin FTSE Italy ETF',
  'NQ=F': 'Nasdaq 100 June 2020 Futures',
  '^DJI': 'Dow Jones Industrial Average',
  '^FCHI': 'CAC 40 France',
  '^GDAXI': 'DAX Frankfurt',
  '^GSPC': 'S&P 500',
  '^HSI': 'Hang Seng Index Hong Kong',
  '^N225': 'Nikkei 225 Japan',
  'ASHR': 'Xtrackers Harvest CSI 300 China A-Shares ETF',
    '^FTSE': 'FTSE 100 London',
}
print(STOCKS.values())

dict_values(['OMX Stockhlom', 'OMX Helsinki', 'Oslo Børs All-share Index', 'OMX Copenhagen', 'iShares China Large-Cap ETF', 'Bolsa de Madrid 35', 'VanEck Vectors Africa ETF', 'Bitcoin/USD', 'Oil Crude Jun 2020 Futures', 'CNY/EUR', 'CNY/USD', 'Franklin FTSE Italy ETF', 'Nasdaq 100 June 2020 Futures', 'Dow Jones Industrial Average', 'CAC 40 France', 'DAX Frankfurt', 'S&P 500', 'Hang Seng Index Hong Kong', 'Nikkei 225 Japan', 'Xtrackers Harvest CSI 300 China A-Shares ETF', 'FTSE 100 London'])


In [3]:
max_date = datetime.today().date()
min_date = max_date - timedelta(days=901)
print("From", min_date,"to", max_date)

From 2017-11-25 to 2020-05-14


In [4]:
selected_features = ["stock", "Date", "Adj Close"]
df_ori= pd.DataFrame.from_dict({}, orient='index', columns = selected_features)
for key, name in STOCKS.items():
    temp_df = yf.download(key, 
                          start=min_date, 
                          end=max_date, 
                          progress=False,
                         interval = "1d")
    if not temp_df.empty:
        temp_df = temp_df.reset_index()
        temp_df["stock"] = name
        df_ori = pd.concat([df_ori, temp_df[selected_features]])
STOCKS = sorted(df_ori.stock.unique())

In [5]:
print("Downloaded Indexes:", df_ori.stock.unique())

Downloaded Indexes: ['OMX Stockhlom' 'OMX Helsinki' 'Oslo Børs All-share Index'
 'OMX Copenhagen' 'iShares China Large-Cap ETF' 'Bolsa de Madrid 35'
 'VanEck Vectors Africa ETF' 'Bitcoin/USD' 'Oil Crude Jun 2020 Futures'
 'CNY/EUR' 'CNY/USD' 'Franklin FTSE Italy ETF'
 'Nasdaq 100 June 2020 Futures' 'Dow Jones Industrial Average'
 'CAC 40 France' 'DAX Frankfurt' 'S&P 500' 'Hang Seng Index Hong Kong'
 'Nikkei 225 Japan' 'Xtrackers Harvest CSI 300 China A-Shares ETF'
 'FTSE 100 London']


In [6]:
df_ori.head()

,stock,Date,Adj Close
0,OMX Stockhlom,2017-11-27,1608.739990
1,OMX Stockhlom,2017-11-28,1620.810059
2,OMX Stockhlom,2017-11-29,1613.560059
3,OMX Stockhlom,2017-11-30,1610.109985
4,OMX Stockhlom,2017-12-01,1592.199951


<a id="proc"/>

## Data processing
> Once the data are downloaded, we are going to perform the following actions:
  - column rename
  - change columns' data type
  - filter out dates which are not in common within all the stocks
  - calculate the normalized adj_close (within a stock)
  - calculate the adj_close increase percentage (in comparison with the previous day)

Column rename

In [7]:
df_ori = df_ori.rename(columns={name: name.lower().replace(" ", "_") for name in df_ori.columns.to_list()})
selected_features = df_ori.columns.to_list()
selected_features

['stock', 'date', 'adj_close']

In [8]:
df_stock = df_ori.copy()

Change columns' data type

In [9]:
df_stock["date"] = pd.to_datetime(df_stock["date"])
df_stock.dtypes

stock                object
date         datetime64[ns]
adj_close           float64
dtype: object

Remove nan values and sorting the dataframe

In [10]:
df_stock = df_stock.dropna()
df_stock = df_stock.sort_values(by=["stock", "date"])
df_stock.shape

(13274, 3)

Filter out dates which are not in common within all the stocks

In [11]:
# Here we select a common interval within the stocks
min_date = max([min(df_stock[df_stock.stock == stock].date) for stock in STOCKS])
max_date = min([max(df_stock[df_stock.stock == stock].date) for stock in STOCKS])
df_stock = df_stock[(df_stock.date >= min_date) & (df_stock.date <= max_date)]
df_stock.tail(5)

,stock,date,adj_close
615,iShares China Large-Cap ETF,2020-05-07,38.200001
616,iShares China Large-Cap ETF,2020-05-08,39.070000
617,iShares China Large-Cap ETF,2020-05-11,38.889999
618,iShares China Large-Cap ETF,2020-05-12,38.549999
619,iShares China Large-Cap ETF,2020-05-13,38.669998


In [12]:
def get_all_dates_list(min_date, max_date):
    temp = min_date
    dates = []
    while temp <= max_date:
        dates.append(temp)
        temp += timedelta(days=1)
    return dates

In [13]:
def get_stock_with_all_dates(stock, df, range_dates):
    temp_df = df[df.stock == stock]
    temp_df = temp_df.set_index("date").reindex(range_dates).sort_index()
    temp_df = temp_df.ffill()
    return temp_df

In [14]:
df_stock[df_stock.stock == STOCKS[0]].sort_values(by='date').head(10)

,stock,date,adj_close
24,Bitcoin/USD,2017-12-18,19114.199219
25,Bitcoin/USD,2017-12-19,17776.699219
26,Bitcoin/USD,2017-12-20,16624.599609
27,Bitcoin/USD,2017-12-21,15802.900391
28,Bitcoin/USD,2017-12-22,13831.799805
29,Bitcoin/USD,2017-12-23,14699.200195
30,Bitcoin/USD,2017-12-24,13925.799805
31,Bitcoin/USD,2017-12-25,14026.599609
32,Bitcoin/USD,2017-12-26,16099.799805
33,Bitcoin/USD,2017-12-27,15838.500000


In [15]:
get_stock_with_all_dates(STOCKS[0], df_stock, get_all_dates_list(min(df_stock.date), max(df_stock.date))).head(10)

,stock,adj_close
date,,
2017-12-18,Bitcoin/USD,19114.199219
2017-12-19,Bitcoin/USD,17776.699219
2017-12-20,Bitcoin/USD,16624.599609
2017-12-21,Bitcoin/USD,15802.900391
2017-12-22,Bitcoin/USD,13831.799805
2017-12-23,Bitcoin/USD,14699.200195
2017-12-24,Bitcoin/USD,13925.799805
2017-12-25,Bitcoin/USD,14026.599609
2017-12-26,Bitcoin/USD,16099.799805


In [16]:
def get_all_stocks_with_all_dates(dataframe_to_normalize):
    df = dataframe_to_normalize.copy()
    stocks = sorted(df.stock.unique())
    range_dates = get_all_dates_list(min(df.date), max(df.date))
    new_df = get_stock_with_all_dates(stocks[0], df, range_dates)
    for stock in stocks[1:]:
        new_df = new_df.append(get_stock_with_all_dates(stock, df, range_dates))
    new_df = new_df.reset_index()
    new_df = new_df.rename(columns={"index": "date"})
    return new_df 

The stock dataset has missing dates given the closure of the market as well as other factors. For this reason, we insert those dates with null value on the other columns. Consequently, we fill the null values with the amount of the previous day (within a stock).

In [17]:
df_stock = get_all_stocks_with_all_dates(df_stock)

In [18]:
df_stock.head(1)

,date,stock,adj_close
0,2017-12-18,Bitcoin/USD,19114.199219


Calculate the normalized `adj_close` (within a stock) and the `adj_close` increase percentage (in comparison with the previous day)

In [19]:
float_features = df_stock.columns[(df_stock.dtypes.values == np.dtype('float64'))].to_list()
print("Features", selected_features, " of which Float:", float_features)
selected_features += [f+"_norm" for f in float_features]

Features ['stock', 'date', 'adj_close']  of which Float: ['adj_close']


In [20]:
df_stock_perc = pd.DataFrame.from_dict({}, orient='index', columns = selected_features)
for stock in STOCKS:
    #Selecting one stock
    temp = df_stock[df_stock.stock == stock].sort_values(by="date",  ascending=True)
    
    # MinMax Scaler
    min_value = min(temp[float_features].min())
    diff_max_min = max(temp[float_features].max()) - min_value
    for feature in float_features:
        temp[feature+"_norm"] = (temp[feature] - min_value) / diff_max_min
    
    #Calculating percentage diff
    temp[float_features] = temp[float_features].pct_change()*100
    
    df_stock_perc = pd.concat([df_stock_perc,temp], sort=False)
df_stock_perc = df_stock_perc.rename(columns={name: name+"_diff%" for name in float_features})
df_stock_perc = df_stock_perc.drop(columns=["stock", "date"])

In [21]:
df_stock = pd.merge(df_stock, df_stock_perc.dropna(), how='inner', left_index=True, right_index=True)
df_stock.head(5)

,date,stock,adj_close,adj_close_diff%,adj_close_norm
1,2017-12-19,Bitcoin/USD,17776.699219,-6.997416,0.915761
2,2017-12-20,Bitcoin/USD,16624.599609,-6.480953,0.843199
3,2017-12-21,Bitcoin/USD,15802.900391,-4.942671,0.791446
4,2017-12-22,Bitcoin/USD,13831.799805,-12.473031,0.667302
5,2017-12-23,Bitcoin/USD,14699.200195,6.271059,0.721933


<a id="stats"/>

## Dataset's stats

In [22]:
get_number_unique_values = lambda df, col: len(df[col].unique())
def print_info(df, main_feature):
    print("Number of %s:" % main_feature, get_number_unique_values(df, main_feature))
    print("\nNumber of days:", get_number_unique_values(df, "date"))
    print("\nFrom:", min(df.date), "to", max(df.date))
    features = df.columns.to_list()
    print("\nWe have %d features:" % len(features), features)
    print("\nThe total number of (rows, cols) is:", df.shape)
    print("\nIn memory occupies: ~%d MB\n" % (df.memory_usage(index=True).sum() / (2**16)))
    print(df.head(1))

### Original Dataset

In [23]:
print_info(df_ori, "stock")

Number of stock: 21

Number of days: 902

From: 2017-11-24 00:00:00 to 2020-05-13 00:00:00

We have 3 features: ['stock', 'date', 'adj_close']

The total number of (rows, cols) is: (13274, 3)

In memory occupies: ~6 MB

           stock       date   adj_close
0  OMX Stockhlom 2017-11-27  1608.73999


### Processed Dataset

In [24]:
print_info(df_stock, "stock")

Number of stock: 21

Number of days: 877

From: 2017-12-19 00:00:00 to 2020-05-13 00:00:00

We have 5 features: ['date', 'stock', 'adj_close', 'adj_close_diff%', 'adj_close_norm']

The total number of (rows, cols) is: (18417, 5)

In memory occupies: ~13 MB

        date        stock     adj_close  adj_close_diff%  adj_close_norm
1 2017-12-19  Bitcoin/USD  17776.699219        -6.997416        0.915761


In [25]:
df_stock.describe()

,adj_close,adj_close_diff%,adj_close_norm
count,18417.000000,18417.000000,18417.000000
mean,6454.170283,-0.032602,0.568072
std,8497.247854,3.692908,0.232318
min,-2.720000,-433.088247,0.000000
25%,38.937679,-0.230575,0.393732
50%,2826.060059,0.000000,0.597784
75%,8784.494141,0.325337,0.749305
max,33154.121094,48.401042,1.000000


In [26]:
df_stock.to_csv("stocks.csv")

In [27]:
dict_country_date_unix = defaultdict(dict)
dict_country_date = defaultdict(dict)
for (stock, date), row in df_stock[df_stock.date >= datetime(2020,1,1)].set_index(["stock", "date"]).to_dict(orient='index').items():
    dict_country_date_unix[stock][str(int(date.timestamp()*1000))] = row
    dict_country_date[stock][str(date.date())] = row

In [28]:
with open('datastock_country_date_unix.json', 'w') as fp:
    json.dump(dict_country_date_unix, fp)
with open('datastock_country_date.json', 'w') as fp:
    json.dump(dict_country_date, fp)

<a id="graphs"/>

#### Come back to the [Back to the top](#top)

#### Come back to [Home](FinalProjectReport.ipynb)